In [552]:
import pandas as pd
import re
pd.set_option('display.max_rows' , 200)

In [553]:
company_name = 'tvs'.upper()

In [554]:
particulars = 'particulars'

In [555]:
share_price_in = 1000000;

In [556]:
def calculate_growth(current , previous) :
    if current == 0 and previous == 0 or current == 0 and previous != 0:
        return 0
    if current != 0 and previous == 0 :
        return current / 100
    return ((current/previous) - 1)
       

In [557]:
def calculate_growth_series(curr_df,prev_df):
    temp_df = curr_df.copy()
    for index in curr_df.index :
        temp_df.loc[index] = calculate_growth(curr_df.loc[index],prev_df[index])
    return temp_df
    

In [558]:
def get_share_price_in(price) :
    return round((price / share_price_in),2)

    

In [559]:
def vertical_analysis(item , divider) :
    return item/divider
    

In [560]:
def round_off_to(num , limit) :
    return round(num,limit)

In [561]:
def count_leading_space(s): 
    match = re.search(r"^\s*", s) 
    return 0 if not match else match.end()


In [562]:
def getColumnFormatterbyPercentage(type) :
    columns_perc_formats = {}
    if type == 'income_statement' :
        for column in income_statement.columns[1:] :
            columns_perc_formats[column] = '{:,.2%}'
    elif type == 'balance_sheet' :
        for column in balance_sheet.columns[1:] :
            columns_perc_formats[column] = '{:,.2%}'
    elif type == 'cashflow_statement' :
        for column in cashflow_statement.columns[1:] :
            columns_perc_formats[column] = '{:,.2%}'
    return columns_perc_formats

In [563]:
def styles(color = 'white',backcolor = '#308D46'):
    return [dict(selector="caption",
                       props=[("text-align", "center"),
                              ("font-size", "150%"),
                              ("color", color),
                              ("background", backcolor),
                              ("font-weight", '600'),
                              ("text-transform", 'uppercase')
                             ])]

In [564]:
income_statement = pd.read_excel('data/'+company_name+'/'+company_name+'-income.xls')
balance_sheet = pd.read_excel('data/'+company_name+'/'+company_name+'-balance.xls')
cashflow_statement = pd.read_excel('data/'+company_name+'/'+company_name+'-cashflow.xls')

In [565]:
start_col = income_statement.columns[1];
start_col_balance_sheet = balance_sheet.columns[1];
start_col_cashflow = cashflow_statement.columns[1];

In [566]:
income_statement_temp_columns = income_statement.columns.values
income_statement_temp_columns[0] = particulars
income_statement.columns = income_statement_temp_columns

balance_sheet_temp_columns = balance_sheet.columns.values
balance_sheet_temp_columns[0] = particulars
balance_sheet.columns = balance_sheet_temp_columns

cashflow_statement_temp_columns = cashflow_statement.columns.values
cashflow_statement_temp_columns[0] = particulars
cashflow_statement.columns = cashflow_statement_temp_columns

In [567]:
basic_eps_filter = income_statement[particulars].eq('Basic EPS')
basic_eps_pos = income_statement[basic_eps_filter].index[0]

In [568]:
WASO_filter = income_statement[particulars].isin(['Basic Weighted Average Shares Outstanding','Diluted Weighted Average Shares Outstanding','Basic WASO','Diluted WASO'])
waso_pos = income_statement[WASO_filter].index

In [569]:
income_statement.dropna(how='all',subset=income_statement.columns[1:], inplace=True)
balance_sheet.dropna(how='all',subset=balance_sheet.columns[1:], inplace=True)
cashflow_statement.dropna(how='all',subset=cashflow_statement.columns[1:], inplace=True)
income_statement.fillna(0 , inplace=True)
balance_sheet.fillna(0 , inplace=True)
cashflow_statement.fillna(0 , inplace=True)

In [570]:
temp_df = income_statement.loc[0:basic_eps_pos,start_col:].copy()
temp_df = pd.DataFrame(temp_df)


In [571]:
temp_df = temp_df.apply(get_share_price_in)

In [572]:
income_statement.loc[0:basic_eps_pos,start_col:] = temp_df

In [573]:
income_statement.loc[waso_pos,start_col:] = income_statement.loc[waso_pos,start_col:].apply(get_share_price_in)

In [574]:
temp_revenue = income_statement.loc[1]
income_statement.loc[1] = income_statement.loc[0]
income_statement.loc[0] = temp_revenue

In [575]:
total_rev = income_statement[particulars].index == 0
income_statement.loc[total_rev,particulars] = income_statement.loc[total_rev,particulars].values[0].strip() 

In [576]:
is_highlighted_indexes = pd.DataFrame(income_statement.loc[0:basic_eps_pos - 1,particulars])
is_highlighted_indexes = is_highlighted_indexes[particulars].apply(count_leading_space).isin([0,4])
is_highlighted_indexes = is_highlighted_indexes.loc[is_highlighted_indexes].index

In [577]:
balance_sheet.loc[0:,start_col_balance_sheet:] =  balance_sheet.loc[0:,start_col_balance_sheet:].applymap(get_share_price_in)

In [578]:
total_equity_filter = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Total Equity')
total_equity_row = balance_sheet.loc[total_equity_filter,start_col_balance_sheet:].index.values[0]
total_assets_filter = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Total Assets')
total_assets_row = balance_sheet.loc[total_assets_filter,start_col_balance_sheet:]
total_liabilities_filter = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Total Liabilities')
total_liabilities_row = balance_sheet.loc[total_liabilities_filter,start_col_balance_sheet:].index.values[0]
balance_sheet.loc[total_liabilities_filter,start_col_balance_sheet:] = balance_sheet.loc[[total_equity_row,total_liabilities_row],start_col_balance_sheet:].sum().values

In [579]:
cashflow_statement.loc[0:,start_col_cashflow:] =  cashflow_statement.loc[0:,start_col_cashflow:].applymap(get_share_price_in)

In [580]:
vertical_analysis_is = pd.DataFrame(columns=income_statement.columns) 
vertical_analysis_is[particulars] = income_statement.loc[0:basic_eps_pos - 1,particulars].copy()

In [581]:
total_revenue_filter = income_statement[particulars].apply(lambda x: x.strip()).eq('Total Revenue')
total_revenue = income_statement.loc[total_revenue_filter,start_col:]

In [582]:
for column in vertical_analysis_is.columns[1:] : 
    vertical_analysis_is[column] = income_statement.loc[0:basic_eps_pos - 1,column].apply(vertical_analysis,divider=total_revenue[column])
vertical_analysis_is.loc[0,particulars] = vertical_analysis_is.loc[0,particulars].strip() 

In [583]:
def row_style(row , list , key , color,bgcolor):
   if row[key] in list :
       return pd.Series('color:'+color+';background-color:'+bgcolor, row.index)
   else :
        return pd.Series('', row.index)

In [584]:
fil = vertical_analysis_is.loc[0:basic_eps_pos - 1,particulars].apply(count_leading_space).eq(0)
templist = vertical_analysis_is.loc[fil,particulars].values
vertical_analysis_is_style = vertical_analysis_is
vertical_analysis_is_style = vertical_analysis_is_style.loc[is_highlighted_indexes].style.apply(row_style,list=templist,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " income statement vertical analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('income_statement'),precision=2) 
fil = None
templist = None

In [585]:
balance_sheet_va_results_till = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Non-Controlling/Minority Interests in Equity')
balance_sheet_va_results_till = balance_sheet[balance_sheet_va_results_till].index.values[0]
bs_va_temp = balance_sheet.loc[:balance_sheet_va_results_till]
bs_va_temp = pd.DataFrame(bs_va_temp)
filt = bs_va_temp[particulars].apply(count_leading_space).isin([0,4,8])
balance_sheet_va = pd.DataFrame(bs_va_temp[filt])
bs_va_temp = None
balance_sheet_va_results_till = None
filt = None

In [586]:
for column in balance_sheet_va.columns[1:] :
    balance_sheet_va[column] = balance_sheet_va.loc[:,column].apply(vertical_analysis,divider=total_assets_row[column])

In [587]:
bs_fil = balance_sheet_va[particulars].apply(count_leading_space).isin([0,4])
bs_templist = balance_sheet_va.loc[bs_fil,particulars].values
balance_sheet_va_style = balance_sheet_va
balance_sheet_va_style = balance_sheet_va_style.style.apply(row_style,list=bs_templist,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " balance sheet vertical analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('balance_sheet'),precision=2) 
bs_fil = None
bs_templist = None

In [588]:
vertical_analysis_is_style

,particulars,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,TTM
0,Total Revenue,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%
1,Gross Profit,28.49%,29.55%,28.50%,29.43%,27.80%,27.25%,24.27%,26.52%,24.87%,23.47%,26.60%
5,Cost of Revenue,-71.51%,-70.45%,-71.50%,-70.57%,-72.20%,-72.75%,-75.73%,-73.48%,-75.13%,-76.53%,-73.40%
7,Operating Income/Expenses,-24.77%,-25.51%,-24.38%,-24.63%,-23.05%,-23.90%,-22.72%,-27.09%,-26.81%,-25.50%,-24.55%
8,"Selling, General and Administrative Expenses",-16.75%,-6.46%,-16.71%,-17.03%,-15.67%,-16.06%,-14.54%,-16.36%,-15.37%,-15.20%,-13.85%
15,Operation and Maintenance Expenses,-0.99%,0.00%,-0.64%,-0.66%,-0.65%,-0.52%,-0.43%,-0.58%,-0.53%,-0.47%,0.00%
16,"Depreciation, Amortization and Depletion",-2.37%,-1.78%,-1.73%,-1.88%,-2.54%,-2.39%,-2.38%,-3.30%,-3.27%,-3.48%,-3.03%
20,Provisions,0.00%,0.00%,0.00%,0.00%,0.00%,-0.52%,-0.99%,-1.63%,-2.70%,-2.60%,0.00%
22,"Other Income/Expense, Operating",-4.66%,-17.28%,-5.31%,-5.07%,-4.18%,-4.41%,-4.38%,-5.22%,-4.92%,-3.75%,-5.28%
26,Total Operating Profit/Loss,3.73%,4.03%,4.12%,4.79%,4.75%,3.35%,1.55%,-0.57%,-1.93%,-2.03%,2.05%


In [589]:
balance_sheet_va_style

,particulars,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Total Assets,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%
1,Total Current Assets,40.07%,40.59%,48.10%,44.36%,39.72%,48.23%,49.90%,50.55%,47.91%,47.46%
2,"Cash, Cash Equivalents and Short Term Investments",2.37%,0.00%,0.61%,1.09%,0.84%,1.28%,1.21%,5.60%,7.16%,5.89%
8,"Cash Restricted or Pledged, Current",0.00%,0.00%,0.00%,0.00%,0.00%,0.03%,0.03%,0.14%,0.38%,0.27%
9,Inventories,18.90%,19.32%,22.26%,20.57%,18.96%,7.99%,7.74%,6.16%,6.22%,6.07%
14,"Trade and Other Receivables, Current",16.49%,21.27%,21.89%,20.68%,18.64%,37.83%,40.23%,37.63%,32.37%,34.12%
22,"Prepayments and Deposits, Current",2.14%,0.00%,3.21%,1.91%,1.26%,1.08%,0.67%,0.99%,1.77%,1.02%
23,Other Current Assets,0.17%,0.00%,0.05%,0.04%,0.03%,0.01%,0.03%,0.03%,0.02%,0.10%
24,"Derivative Investment and Hedging Assets, Current",0.00%,0.00%,0.08%,0.07%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
25,Total Non-Current Assets,59.93%,59.41%,51.90%,55.64%,60.28%,51.77%,50.10%,49.45%,52.09%,52.62%


In [590]:
income_statement_ha = pd.DataFrame(columns= income_statement.columns)

In [591]:
income_statement_ha[particulars] = income_statement[particulars]

In [592]:
income_statement_ha.fillna(0 , inplace=True)

In [593]:
temp_ha_filt = income_statement[particulars].apply(count_leading_space).isin([0,4])
for index , column in enumerate(income_statement_ha.columns) :
    if index < 2 :
        pass;
    else:
        prevColumn = income_statement.columns[index-1]
        income_statement_ha.loc[temp_ha_filt,column] = calculate_growth_series(income_statement.loc[temp_ha_filt,column],income_statement.loc[temp_ha_filt,prevColumn])
        

In [594]:
income_statement_ha = income_statement_ha[temp_ha_filt]

In [595]:
#income_statement_ha.replace([float('inf'), float('-inf')], None, inplace=True)
#income_statement_ha.fillna(0 , inplace=True)

In [596]:
temp_ha_filt = income_statement[particulars].apply(count_leading_space).isin([0])
iha_templist = income_statement_ha.loc[temp_ha_filt,particulars].values
income_statement_ha_style = income_statement_ha
income_statement_ha_style = income_statement_ha.style.apply(row_style,list=iha_templist,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " Income statement horizontal analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('income_statement'),precision=2) 

In [597]:
income_statement_ha_style

,particulars,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,TTM
0,Total Revenue,0.00%,13.13%,23.07%,11.68%,8.22%,25.37%,18.57%,-9.11%,2.57%,23.45%,28.95%
1,Gross Profit,0.00%,17.32%,18.70%,15.31%,2.24%,22.89%,5.61%,-0.68%,-3.80%,16.48%,46.16%
5,Cost of Revenue,0.00%,11.47%,24.90%,10.24%,10.71%,26.33%,23.42%,-11.81%,4.87%,25.75%,23.67%
7,Operating Income/Expenses,0.00%,16.56%,17.61%,12.81%,1.27%,29.99%,12.74%,8.37%,1.49%,17.44%,24.13%
8,"Selling, General and Administrative Expenses",0.00%,-56.39%,218.40%,13.86%,-0.43%,28.47%,7.33%,2.27%,-3.59%,22.01%,17.50%
15,Operation and Maintenance Expenses,0.00%,0.00%,-660.40%,14.31%,7.35%,-0.42%,-1.46%,23.70%,-6.33%,9.63%,0.00%
16,"Depreciation, Amortization and Depletion",0.00%,-15.17%,19.89%,21.11%,46.48%,17.92%,18.23%,25.87%,1.59%,31.52%,12.27%
20,Provisions,0.00%,0.00%,0.00%,0.00%,0.00%,-808.60%,127.20%,49.00%,70.53%,18.71%,0.00%
22,"Other Income/Expense, Operating",0.00%,319.77%,-62.21%,6.62%,-10.60%,32.20%,17.68%,8.37%,-3.27%,-6.06%,81.83%
26,Total Operating Profit/Loss,0.00%,22.38%,25.58%,30.09%,7.22%,-11.56%,-45.23%,-133.55%,247.05%,29.69%,-230.41%
